<a href="https://colab.research.google.com/github/dhirs/swingedge_backtesting/blob/main/JSONVisualizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
!pip install boto3
!pip install tabulate
!pip install psycopg2

In [25]:
import boto3, json, psycopg2
import pandas.io.sql as sqlio
import pandas as pd
import base64
from tabulate import tabulate
import warnings
warnings.filterwarnings('ignore')

from google.colab import userdata
key = userdata.get('AWS_KEY')
secret = userdata.get('AWS_SECRET')

region = "ap-south-1"
ssm = boto3.client('ssm', aws_access_key_id=key,  aws_secret_access_key=secret, region_name=region)
parameter = ssm.get_parameter(Name='timescaledb_credentials', WithDecryption=True)
config = json.loads(parameter['Parameter']['Value'])

conn = psycopg2.connect(database = config['database'],
                         user =  config['user'],
                         password = config['password'],
                         host = config['host'],
                         port = config['port'],
        )

In [26]:
def get_data(test_id, node_name):
  query = f"select * from backtests where test_id = {test_id}"
  data = sqlio.read_sql_query(query,conn)
  df = pd.DataFrame(data)
  val = df['payload_best'][0]
  base64_bytes = val.encode("ascii")
  sample_string_bytes = base64.b64decode(base64_bytes)
  json_payload = sample_string_bytes.decode("ascii")
  data = json.loads(json_payload)
  # print(data)
  return data[0][node_name],data

In [35]:
test_id = 103

In [17]:
sharpe,all = get_data(test_id, 'sharpe')
print(sharpe)
# print(all)

{'sharperatio': -87.2703975428}


In [ ]:
returns,all = get_data(test_id, 'returns')
df = pd.DataFrame([returns])
print(df)

       rtot      ravg     rnorm  rnorm100
0  0.004699  0.000047  0.012034  1.203392


In [ ]:
drawdown,all = get_data(test_id, 'drawdown')
# df = pd.DataFrame([drawdown])
pd.json_normalize(drawdown)
# print(df)

,len,drawdown,moneydown,max.len,max.drawdown,max.moneydown
0,232,0.556943,56.4353,232,0.627603,63.5953


In [94]:
def get_trades(test_id):

  trades,all= get_data(test_id, 'trades')
  df = pd.json_normalize(trades)
  df_all = pd.json_normalize(all)
  df_final = pd.DataFrame(
      [
        [
            all[0]['timeframe'] ,
            all[0]['max_loss_p'],
            all[0]['risk_reward'],

            df['won.total'][0]/(df['won.total'][0]+df['lost.total'][0]),

            df['won.pnl.total'][0]+df['lost.pnl.total'][0],

            100*df['long.won'][0]/(   df['long.won'][0]  +df['long.lost'][0]  ),

            100*df['short.won'][0]/(   df['short.won'][0]  +df['short.lost'][0]),
            df['len.average'][0]

        ]

      ],
      columns=['Timeframe','Sharpe','MaxLossP','RiskReward','WinRatio',
               'NetRevenue',
               'LongWinP',
               'ShortWinP',
               'AvTime'])
  return df_final
  # print(df.columns)
  # print(df_all)
  # print(f"Timeframe:{all[0]['timeframe']}")
  # print(f"Total wins:{df['won.total'][0]}")
  # print(f"Total losses:{df['lost.total'][0]}")
  # win_ratio = df['won.total'][0]/(df['won.total'][0]+df['lost.total'][0])
  # print(f"Win ratio:{win_ratio}")
  # print(f"Gross revenue:{df['won.pnl.total'][0]}")
  # print(f"Gross loss:{df['lost.pnl.total'][0]}")
  # print(f"Net revenue:{df['won.pnl.total'][0]+df['lost.pnl.total'][0]}")
  # print(f"Long wins:{df['long.won'][0]}")
  # print(f"Long losses:{df['long.lost'][0]}")
  # print(f"Long win%:{100*df['long.won'][0]/(   df['long.won'][0]  +df['long.lost'][0]  )  })")
  # print(f"Short wins:{df['short.won'][0]}")
  # print(f"Short losses:{df['short.lost'][0]}")
  # print(f"Short win%:{100*df['short.won'][0]/(   df['short.won'][0]  +df['short.lost'][0]  )  })")
  # print(f"Avg. trade length:{df['len.average'][0]}")

In [ ]:
transactions,all = get_data(test_id, 'transactions')
# print(transactions)
table_data = []
for timestamp, transactions_list in transactions.items():
    for index, transaction in enumerate(transactions_list):
        row = [timestamp, index] + transaction
        table_data.append(row)
headers = ["Timestamp", "Transaction ID", "0", "1", "2", "3", "4"]
print(tabulate(table_data, headers=headers, tablefmt="grid"))

+---------------------+------------------+-----+---------+-----+-----+----------+
| Timestamp           |   Transaction ID |   0 |       1 |   2 | 3   |        4 |
+=====================+==================+=====+=========+=====+=====+==========+
| 2024-10-08 12:30:00 |                0 |   1 | 245.16  |   0 |     | -245.16  |
+---------------------+------------------+-----+---------+-----+-----+----------+
| 2024-10-09 14:30:00 |                0 |  -1 | 243.44  |   0 |     |  243.44  |
+---------------------+------------------+-----+---------+-----+-----+----------+
| 2024-10-14 15:30:00 |                0 |   1 | 218.96  |   0 |     | -218.96  |
+---------------------+------------------+-----+---------+-----+-----+----------+
| 2024-10-21 11:30:00 |                0 |  -1 | 217.76  |   0 |     |  217.76  |
+---------------------+------------------+-----+---------+-----+-----+----------+
| 2024-10-22 15:30:00 |                0 |   1 | 217.85  |   0 |     | -217.85  |
+---------------

In [92]:
from IPython.display import display

def generate_combined_results(strategy_id,sort_field):
  query = f"select test_id, symbol,payload_best from backtests where strategy_id = {strategy_id}"
  data = sqlio.read_sql_query(query,conn)
  df = pd.DataFrame(data)
  frames = []
  for index, row in df.iterrows():
      test_id = row['test_id']
      stats = get_trades(test_id)
      stats.insert(0, 'Symbol', row['symbol'])
      frames.append(stats)
  final = pd.concat(frames)
  display(final.sort_values(by=sort_field, ascending=[False]))

In [93]:
strategy_id = 1
sort_field = 'NetRevenue'
generate_combined_results(strategy_id,sort_field)


,Symbol,Timeframe,MaxLossP,RiskReward,WinRatio,NetRevenue,LongWinP,ShortWinP,AvTime
0,NFLX,1d,1,7,1.000000,156.1650,100.000000,100.000000,16.000000
0,ADBE,1h,1,5,0.214286,85.2699,20.000000,22.222222,13.500000
0,TSLA,1h,1,3,0.407407,76.6124,40.000000,41.666667,5.296296
0,IBM,4h,1,5,0.600000,51.1842,66.666667,50.000000,16.200000
0,META,4h,1,2,0.500000,44.6889,75.000000,25.000000,2.750000
0,MSFT,1h,1,5,0.250000,44.4545,33.333333,20.000000,29.812500
0,AMZN,1h,1,7,0.214286,27.6486,33.333333,12.500000,29.000000
0,GOOGL,1h,1,2,0.416667,16.6242,50.000000,35.714286,7.708333
0,INTC,4h,1,2,0.888889,6.4738,100.000000,66.666667,2.444444
